In [38]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
%matplotlib inline
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
from case_study_trains import *
import case_study_trains
# import pyomo.util.infeasible as infeas
# print(infeas.log_infeasible_bounds(m))
# infeas.log_close_to_bounds(m)


m = wt.watertap_setup(dynamic = False)

# wt.case_study_trains.train = {"case_study": "KBHDP",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "Irwin",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "EMWD",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "Santa_Barbara",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

wt.case_study_trains.train = {"case_study": "tampa_bay",
                             "reference": "nawi",
                             "scenario": "baseline"}

# wt.case_study_trains.train = {"case_study": "Ashkelon",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "Tampa_Bay",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "HRSD",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "Big_Spring",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "OCWD",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "Solaire",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}


# TODO LATER: how to make this sync with info in train input data. We might not need to do that.
#But, if the source water type is different to what is in the train (pfd dictionary), 
#then we should updat the node name. If more than two sources - what to do? Needs to be
#based on pfd node!?

# wt.case_study_trains.source_water = {"case_study": "KBHDP", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "KBHDP_Brackish_Ave"}

# wt.case_study_trains.source_water = {"case_study": "Irwin", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "Brackish"}

# wt.case_study_trains.source_water = {"case_study": "EMWD", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "EMWD_CA_Brackish"}

# wt.case_study_trains.source_water = {"case_study": "Santa_Barbara", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "Seawater"}

wt.case_study_trains.source_water = {"case_study": "tampa_bay", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": "seawater"}

# wt.case_study_trains.source_water = {"case_study": "Ashkelon", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "Seawater"}

# wt.case_study_trains.source_water = {"case_study": "Tampa_Bay", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "Seawater"}
# 
# wt.case_study_trains.source_water = {"case_study": "HRSD", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "HRSD_Municipal"}

# wt.case_study_trains.source_water = {"case_study": "Big_Spring", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "Big_Spring_Feed"}

# wt.case_study_trains.source_water = {"case_study": "OCWD", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "OCWD_Feed"}

# wt.case_study_trains.source_water = {"case_study": "Solaire", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": ["Solaire_Graywater", "Solaire_Blackwater", "Solaire_Stormwater"]}


m = wt.case_study_trains.get_case_study(m=m) # flow is set as case study flow unless defined.


tampa_bay
------- Adding Unit Processes -------
sw_onshore_intake
sulfuric_acid_addition
ferric_chloride_addition
chlorination
static_mixer
tri_media_filtration
cartridge_filtration
ro_first_pass
ro_second_pass
lime_softening
chlorination_b
caustic_soda_addition
ammonia_addition
treated_storage
21600.0
backwash_solids_handling
municipal_drinking
surface_discharge
-------------------------------------
adding source
adding splitter
params into splitter --> [0.67, 0.33]
----- Connecting Unit Processes -----
seawater ToUnitName --> sw_onshore_intake
sw_onshore_intake ToUnitName --> sulfuric_acid_addition
sulfuric_acid_addition ToUnitName --> ferric_chloride_addition
ferric_chloride_addition ToUnitName --> chlorination
chlorination ToUnitName --> static_mixer
tri_media_filtration ToUnitName --> cartridge_filtration
tri_media_filtration ToUnitName --> backwash_solids_handling
lime_softening ToUnitName --> chlorination_b
chlorination_b ToUnitName --> caustic_soda_addition
caustic_soda_additio

In [39]:
wt.display.show_train2(model_name=m)

In [40]:
# RUN MODEL with optimal ro so that the model solves. Then runs again with set pressure. 

In [41]:
# # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("unfixing feed presure and area for", key)
        
wt.run_water_tap(m = m, objective=True)

# prints RO results
print("optimal ro area and pressures:")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

# RESET PRESSURE TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        if "feed_pressure" in m.fs.pfd_dict[key]["Parameter"]:
            if m.fs.pfd_dict[key]["Parameter"]["type"] == "pass":
                getattr(m.fs, key).feed.pressure.fix(m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])
                print("setting feed presure for", key, "to -->", m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])

wt.run_water_tap(m = m, objective=True, print_model_results="summary")

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

unfixing feed presure and area for ro_first_pass
unfixing feed presure and area for ro_second_pass
degrees_of_freedom: 4
WaterTAP3 solution optimal
optimal ro area and pressures:
ro_first_pass feed pressure 58.34633458170394
ro_first_pass membrane area 149292.29208671325
ro_second_pass feed pressure 89.9999984749248
ro_second_pass membrane area 21315.732816039417
setting feed presure for ro_first_pass to --> 85
setting feed presure for ro_second_pass to --> 35
degrees_of_freedom: 2
WaterTAP3 solution optimal
fs.sw_onshore_intake
total_cap_investment: 10.216867778290633
----------------------------------------------------------------------
fs.sulfuric_acid_addition
total_cap_investment: 0.23072715211599915
----------------------------------------------------------------------
fs.ferric_chloride_addition
total_cap_investment: 2.9536394454284336
----------------------------------------------------------------------
fs.chlorination
total_cap_investment: 9.265881440407075
------------------

In [42]:
# If you need the system recovery to match better.... set a maximum recovery rate.

In [43]:
from pyomo.environ import Constraint
m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery <= 0.55) # THIS IS FOR TAMPA BAY
#m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery <= 0.50) # THIS IS FOR SANTA BARBARA

In [44]:
# set cap utilization factor
m.fs.costing_param.plant_cap_utilization = 0.75

In [45]:
wt.run_water_tap(m = m, objective=True, print_model_results="summary")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

degrees_of_freedom: 2
WaterTAP3 solution optimal
fs.sw_onshore_intake
total_cap_investment: 10.216867778290633
----------------------------------------------------------------------
fs.sulfuric_acid_addition
total_cap_investment: 0.23072715211599915
----------------------------------------------------------------------
fs.ferric_chloride_addition
total_cap_investment: 2.9536394454284336
----------------------------------------------------------------------
fs.chlorination
total_cap_investment: 9.265881440407075
----------------------------------------------------------------------
fs.static_mixer
total_cap_investment: 0.1162155841035918
----------------------------------------------------------------------
fs.tri_media_filtration
total_cap_investment: 8.067722349162832
----------------------------------------------------------------------
fs.cartridge_filtration
total_cap_investment: 7.584516536765473
----------------------------------------------------------------------
fs.ro_first_pa

In [46]:
# RESET AREA TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).membrane_area.fix(getattr(m.fs, key).membrane_area[0]())

In [47]:
# Readjust recovery constraint
m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery >= 0)

In [48]:
# NOW RUN AS SIMULATION
wt.run_water_tap(m = m, objective=False, print_model_results="summary")

degrees_of_freedom: 0
WaterTAP3 solution optimal
fs.sw_onshore_intake
total_cap_investment: 10.216867778290633
----------------------------------------------------------------------
fs.sulfuric_acid_addition
total_cap_investment: 0.23072715211599915
----------------------------------------------------------------------
fs.ferric_chloride_addition
total_cap_investment: 2.9536394454284336
----------------------------------------------------------------------
fs.chlorination
total_cap_investment: 9.265881440407075
----------------------------------------------------------------------
fs.static_mixer
total_cap_investment: 0.1162155841035918
----------------------------------------------------------------------
fs.tri_media_filtration
total_cap_investment: 8.067722349162832
----------------------------------------------------------------------
fs.cartridge_filtration
total_cap_investment: 7.584516536765473
----------------------------------------------------------------------
fs.ro_first_pa

In [49]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m = m, case_study = wt.case_study_trains.source_water["case_study"], 
                                scenario = wt.case_study_trains.source_water["scenario"])

In [50]:
# sensitivity analyses
sens_df = pd.DataFrame()

lcow_list = []
water_recovery_list = []
scenario_value = []
scenario_name = []
elec_lcow = []

lcow_list.append(value(m.fs.costing.LCOW))
water_recovery_list.append(value(m.fs.costing.system_recovery))
scenario_value.append("n/a")
scenario_name.append("baseline")
elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))

runs_per_scenario = 50

############ onstream_factor 70-100% ############
stash_value = m.fs.costing_param.plant_cap_utilization
scenario = "plant_cap_utilization"
print(scenario)
ub = 1
lb = 0.7
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub, step):
    m.fs.costing_param.plant_cap_utilization = i
    wt.run_water_tap(m = m, objective=False)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.costing_param.plant_cap_utilization = stash_value    
############################################################    

############ salinity  30-45 ############
stash_value = m.fs.seawater.conc_mass_in[0, "tds"]()
scenario = "inlet salinty 25k-45k mg/L"
print(scenario)
ub = 25
lb = 45
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub, step):
    m.fs.seawater.conc_mass_in[0, "tds"].fix(i) 
    wt.run_water_tap(m = m, objective=False)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.seawater.conc_mass_in[0, "tds"].fix(stash_value) 
############################################################

############ inlet flow ############
stash_value = m.fs.seawater.flow_vol_in[0]()
scenario = "inlet flow"
print(scenario)
ub = stash_value * 1.3
lb = stash_value * 0.7
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub, step):
    m.fs.seawater.flow_vol_in.fix(i) 
    wt.run_water_tap(m = m, objective=False)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.seawater.flow_vol_in.fix(stash_value) 
############################################################

############ lifetime years ############

stash_value = value(m.fs.costing_param.plant_lifetime_yrs)
scenario = "lifetime (yrs)"
print(scenario)
ub = 50
lb = 15
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub, step):
    m.fs.costing_param.plant_lifetime_yrs = i 
    wt.run_water_tap(m = m, objective=False)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.costing_param.plant_lifetime_yrs = stash_value
############################################################

############ elec cost +-20% ############

stash_value = value(m.fs.costing_param.electricity_price)
scenario = "electricity price +- 20%"
print(scenario)
ub = stash_value * 1.3
lb = stash_value * 0.7
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub, step):
    m.fs.costing_param.electricity_price = i 
    wt.run_water_tap(m = m, objective=False)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.costing_param.electricity_price = stash_value
############################################################

############ RO scenarios --> pressure, membrane area, replacement rate% ############

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        area = value(getattr(m.fs, key).membrane_area[0])

        scenario_dict = {"membrane_area" : [-area*0.2, area*0.2], 
                         "pressure": [-10, 10], 
                         "factor_membrane_replacement": [-0.1, 0.1]}
        
        for scenario in scenario_dict.keys():
            if scenario == "pressure":
                stash_value = value(getattr(getattr(getattr(m.fs, key), "feed"), scenario)[0])
            else:
                stash_value = value(getattr(getattr(m.fs, key), scenario)[0])

            print(scenario)
            ub = stash_value + scenario_dict[scenario][1]
            lb = stash_value + scenario_dict[scenario][0]
            step = (ub - lb) / runs_per_scenario
            
            for i in np.arange(lb, ub, step):
                if scenario == "pressure":
                    getattr(getattr(getattr(m.fs, key), "feed"), scenario).fix(i)
                else:
                    getattr(getattr(m.fs, key), scenario).fix(i)
                
                wt.run_water_tap(m = m, objective=False)
                print("LCOW -->", m.fs.costing.LCOW())
    
                lcow_list.append(value(m.fs.costing.LCOW))
                water_recovery_list.append(value(m.fs.costing.system_recovery))
                scenario_value.append(i)
                scenario_name.append(key + "_" + scenario)
                elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
            if scenario == "pressure":
                getattr(getattr(getattr(m.fs, key), "feed"), scenario).fix(stash_value)
            else:
                getattr(getattr(m.fs, key), scenario).fix(stash_value)
                            
############################################################

# final run to get baseline numbers again
wt.run_water_tap(m = m, objective=False)

sens_df["lcow"] = lcow_list
sens_df["water_recovery"] =  water_recovery_list
sens_df["elec_lcow"] =  elec_lcow
sens_df["scenario_value"] = scenario_value
sens_df["scenario_name"] = scenario_name
sens_df["lcow_difference"] =  sens_df.lcow - value(m.fs.costing.LCOW)
sens_df["water_recovery_difference"] = (sens_df.water_recovery - value(m.fs.costing.system_recovery))
sens_df["elec_lcow_difference"] = (sens_df.elec_lcow - value(m.fs.costing.elec_frac_LCOW))

sens_df.elec_lcow = sens_df.elec_lcow * 100
sens_df.water_recovery = sens_df.water_recovery * 100


plant_cap_utilization
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.3055822583116472
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2944866583826526
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.283578062946844
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2728517838692934
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2623032884228627
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.251928192901579
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.241722256546403
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2316813757657044
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.221801578633894
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2120790196527227
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2025099747607275
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1930908365772233
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 

WaterTAP3 solution optimal
LCOW --> 1.180219847052989
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1814394861603084
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1827607997588152
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.184180982708117
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.185697360483698
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1873073851254905
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.189008632034289
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1907987977158356
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1926756983667781
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1946372692783818
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.196681565032388
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.198806760470999
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2010111524289317
degrees_of_freedom: 0
Water

degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.161589322048526
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1663354986353878
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1710816752222502
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.175827851809112
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1805740283959743
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1853202049828362
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1900663815696986
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1948125581565603
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.1995587347434225
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2043049113302844
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2090510879171465
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2137972645040085
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2185434410908706
d

degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2231144765047341
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2220842676418213
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2211118075702476
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2201972712835893
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2193409845291658
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.218543441094571
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2178053227420325
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2171275220778408
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2165111686147847
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2159576582132108
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2154686859471329
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2150462822026673
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2146928514320166


WaterTAP3 solution optimal
LCOW --> 1.2838280497602714
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2773949402051183
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2711197440276416
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2650266761482523
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2591478154648714
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2535250964685216
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2482118545133534
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2432725517883327
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2387783940861383
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2347965518395054
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2313735925866285
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2285196665886762
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.226203314010328
degrees_of_freedom: 0


degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2144089497600046
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.214684582515396
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.214960215270787
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.215235848026178
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2155114807815688
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2157871135369598
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2160627462923512
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2163383790477418
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.216614011803133
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.216889644558524
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2171652773139152
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2174409100693062
degrees_of_freedom: 0
WaterTAP3 solution optimal
LCOW --> 1.2177165428246974
degr

In [51]:
sens_df.to_csv("my_results/tampa_bay_baseline_sensitivity.csv")

In [ ]:
import pyomo.util.infeasible as infeas

In [ ]:
infeas.log_infeasible_bounds(m)

In [ ]:
infeas.log_close_to_bounds(m)

In [ ]:
def it_gets_results(appended, print_it=False, write_it=True):    
    train = wt.case_study_trains.train

    py_var = [
        "fixed_cap_inv",
        "total_cap_investment",
        "cat_and_chem_cost",
        "electricity_cost",
        "total_fixed_op_cost",
    "flow_in", "flow_out"]

    excel_var = ['Fixed Capital Investment (FCI)',
                'Total Capital Investment (TCI)',
                'Catalysts and Chemicals',
                'Electricity',
                'Total Fixed Operating Costs']

    case_study = train['case_study']
    if print_it:
        print(f'\n######### Case study = {case_study} #########\n\n\n')

    cap_inv_tot = []

    big_dict = {}

    for b_unit in m.fs.component_objects(Block, descend_into=True):
        unit = str(b_unit)[3:]
        if hasattr(b_unit, 'costing'):
            if print_it:
                print(f'____________{unit}____________')
            py_vals =  []
            excel_vals = []
            big_dict[unit] = {'python': {}, 'excel': {}}
            df = pd.read_csv("data/case_study_results.csv")
            df = df[df.case_study == train['case_study']]
            df = df[df.scenario == train['scenario']]
            df = df[df.unit_process == unit]
            flow_in_excel = round(df[df.variable == "flow_vol_in"].value.sum(), 5)
            flow_out_excel = round(df[df.variable == 'flow_vol_out'].value.sum(), 5)
            flow_in_python = round(value(b_unit.inlet.flow_vol[0.0] * 3600), 5)
            flow_out_python = round(value(b_unit.outlet.flow_vol[0.0] * 3600), 5)
            waste_python = round(value(b_unit.waste.flow_vol[0.0] * 3600), 5)
            if print_it:
                print(f'\n#### flow_in\nPython --> {flow_in_python}')
                print(f'Excel --> {flow_in_excel}\n\n')
                print(f'#### flow out\nPython --> {flow_out_python}')
                print(f'Excel --> {flow_out_excel}\n\n')
                print(f'#### waste\nPython--> {waste_python}\n')
            for py, ex in dict(zip(py_var, excel_var)).items():
                if print_it:
                    print(f'\n#### {py}')
                try:
                    num = value(getattr(b_unit.costing, py))
                    py_vals.append(num)
                    if print_it:
                        print(f'Python --> {round(num, 5)}')
                except ZeroDivisionError:
                    if print_it:
                        print(f'Python --> {0} - ERROR')
                    py_vals.append(0)

                df = pd.read_csv("data/case_study_results.csv")
                df = df[df.case_study == train['case_study']]
                df = df[df.scenario == train['scenario']]
                df = df[df.unit_process == unit]
                num = df[df.Variable == ex].value.sum()
                if print_it:
                    print(f'Excel --> {round(num, 5)}\n\n')
                excel_vals.append(num)
            py_vals.append(flow_in_python)
            py_vals.append(flow_out_python)
            py_dict = dict(zip(py_var, py_vals))
            excel_vals.append(flow_in_excel)
            excel_vals.append(flow_out_excel)
            excel_dict = dict(zip(py_var, excel_vals))
            big_dict[unit]['python'] = py_dict
            big_dict[unit]['excel'] = excel_dict

    else:
        pass 

    df = pd.DataFrame.from_dict({(i,j): big_dict[i][j] 
                               for i in big_dict.keys() 
                               for j in big_dict[i].keys()},
                               orient='index')

    df['case_study'] = case_study
    df['LCOW'] = value(m.fs.costing.LCOW)
    if write_it:
        df.to_csv(f'results/test_compare_{case_study}_{appended}.csv')

In [ ]:
# wt.run_water_tap(m=m, solver_results=True, print_model_results=True)
# it_gets_results('testy1')
# wt.run_water_tap(m=m, solver_results=True, print_model_results=False)
# it_gets_results('testy2')

In [ ]:
def do_it_all(append1, append2, solver_results=True, model_results=False):
    case_studies = ['Ashkelon', 'Big_Spring', 'Carlsbad', 'EMWD', 'HRSD', 'Irwin', 'KBHDP', 'OCWD', 'Santa_Barbara', 'Solaire', 'Tampa_Bay']
    water_sources = ['Seawater', 'Big_Spring_Feed', 'Seawater', 'EMWD_CA_Brackish', 'HRSD_Municipal', 'Brackish', 'KBHDP_Brackish_Ave', 'OCWD_Feed', 'Seawater', 'Solaire_Graywater', 'Seawater']
    wt_dict = dict(zip(case_studies, water_sources))
    for k, v in wt_dict.items():
        
        m = wt.watertap_setup(dynamic = False)
        
        wt.case_study_trains.train = {"case_study": k,
                                 "reference": "NAWI",
                                 "scenario": "Baseline"}
        
        wt.case_study_trains.source_water = {"case_study": k, 
                                 "reference": "NAWI",
                                 "scenario": "Baseline",
                                 "water_type": v}
        
        wt.run_water_tap(m=m, solver_results=solver_results, print_model_results=model_results)
        
        it_gets_results(append1)
        
        wt.run_water_tap(m=m, solver_results=solver_results, print_model_results=model_results)
        
        it_gets_results(append2)
# value(m.fs.costing.LCOW)

In [ ]:
# wt.run_water_tap(m=m, solver_results=True, print_model_results=False)

In [ ]:
# df.to_csv(f'results/test_compare_{case_study}.csv')
wt.run_water_tap(m=m, solver_results=True, print_model_results=False)

train = wt.case_study_trains.train

py_var = [
    "fixed_cap_inv",
    "total_cap_investment",
    "cat_and_chem_cost",
    "electricity_cost",
    "total_fixed_op_cost",
"flow_in", "flow_out"]

excel_var = ['Fixed Capital Investment (FCI)',
            'Total Capital Investment (TCI)',
            'Catalysts and Chemicals',
            'Electricity',
            'Total Fixed Operating Costs']

case_study = train['case_study']
print(f'\n######### Case study = {case_study} #########\n\n\n')

cap_inv_tot = []

big_dict = {}

for b_unit in m.fs.component_objects(Block, descend_into=True):
    unit = str(b_unit)[3:]
    if hasattr(b_unit, 'costing'):
        print(f'____________{unit}____________')
        py_vals =  []
        excel_vals = []
        big_dict[unit] = {'python': {}, 'excel': {}}
        df = pd.read_csv("data/case_study_results.csv")
        df = df[df.case_study == train['case_study']]
        df = df[df.scenario == train['scenario']]
        df = df[df.unit_process == unit]
        flow_in_excel = round(df[df.variable == "flow_vol_in"].value.sum(), 5)
        flow_out_excel = round(df[df.variable == 'flow_vol_out'].value.sum(), 5)
        flow_in_python = round(value(b_unit.inlet.flow_vol[0.0] * 3600), 5)
        flow_out_python = round(value(b_unit.outlet.flow_vol[0.0] * 3600), 5)
        waste_python = round(value(b_unit.waste.flow_vol[0.0] * 3600), 5)
        print(f'\n#### flow_in\nPython --> {flow_in_python}')
        print(f'Excel --> {flow_in_excel}\n\n')
        print(f'#### flow out\nPython --> {flow_out_python}')
        print(f'Excel --> {flow_out_excel}\n\n')
        print(f'#### waste\nPython--> {waste_python}\n')
        for py, ex in dict(zip(py_var, excel_var)).items():
            print(f'\n#### {py}')
            try:
                num = value(getattr(b_unit.costing, py))
                py_vals.append(num)
                print(f'Python --> {round(num, 5)}')
            except ZeroDivisionError:
                print(f'Python --> {0} - ERROR')
                py_vals.append(0)
            
            df = pd.read_csv("data/case_study_results.csv")
            df = df[df.case_study == train['case_study']]
            df = df[df.scenario == train['scenario']]
            df = df[df.unit_process == unit]
            num = df[df.Variable == ex].value.sum()
            print(f'Excel --> {round(num, 5)}\n\n')
            excel_vals.append(num)
        py_vals.append(flow_in_python)
        py_vals.append(flow_out_python)
        py_dict = dict(zip(py_var, py_vals))
        excel_vals.append(flow_in_excel)
        excel_vals.append(flow_out_excel)
        excel_dict = dict(zip(py_var, excel_vals))
        big_dict[unit]['python'] = py_dict
        big_dict[unit]['excel'] = excel_dict
        
else:
    pass 

df = pd.DataFrame.from_dict({(i,j): big_dict[i][j] 
                           for i in big_dict.keys() 
                           for j in big_dict[i].keys()},
                           orient='index')

df['case_study'] = case_study
df['LCOW'] = value(m.fs.costing.LCOW)
df.to_csv(f'results/test_compare_{case_study}_new2.csv')

In [ ]:
# value(m.fs.costing.LCOW)

In [ ]:
value(m.fs.coag_and_floc.water_recovery[0])

In [ ]:
df.to_csv(f'results/test_compare_{case_study}_new1.csv')

In [ ]:
# df_final = pd.DataFrame()
# for case_name in ["Ashkelon", "Carlsbad", "Tampa_Bay"]: #, "Santa_Barbra"]: #, "Ashkelon"]:

#     m = wt.watertap_setup(dynamic = False)

#     wt.case_study_trains.train = {"case_study": case_name,
#                                  "reference": "NAWI",
#                                  "scenario": "Baseline"}

#     # TODO LATER: how to make this sync with info in train input data. We might not need to do that.
#     #But, if the source water type is different to what is in the train (pfd dictionary), 
#     #then we should updat the node name. If more than two sources - what to do? Needs to be
#     #based on pfd node!?

#     wt.case_study_trains.source_water = {"case_study": case_name, 
#                                  "reference": "NAWI",
#                                  "scenario": "Baseline",
#                                  "water_type": "Seawater"}
    
#     m = wt.case_study_trains.get_case_study(m = m) # flow is set as case study flow unless defined.
    
      
#     wt.run_water_tap(m = m, solver_results = False, print_model_results = False)
#     print(value(m.fs.costing.LCOW))
#     df = get_results_table()
#     df["case_study"] = case_name
#     df_final = pd.concat([df_final,df])
    

In [ ]:
def get_results_table():

    train = wt.case_study_trains.train

    py_var = [
        "fixed_cap_inv",
        "total_cap_investment",
        "cat_and_chem_cost",
        "electricity_cost",
        "total_fixed_op_cost"]

    excel_var = ['Fixed Capital Investment (FCI)',
                'Total Capital Investment (TCI)',
                'Catalysts and Chemicals',
                'Electricity',
                'Total Fixed Operating Costs']
    big_dict = {}
    # for letter in ['A', 'B', 'C']:
    for b_unit in m.fs.component_objects(Block, descend_into=True):
        unit = str(b_unit)[3:]
        if hasattr(b_unit, 'costing'):
            #print(f'____________{unit}____________')
            py_vals =  []
            excel_vals = []
            big_dict[unit] = {'python': {}, 'excel': {}}
            for py, ex in dict(zip(py_var, excel_var)).items():
                #print(f'\n#### {py}')
                try:
                    num = value(getattr(b_unit.costing, py))
                    py_vals.append(num)
                    #print(f'Python --> {num}')
                except ZeroDivisionError:
                    #print(f'Python --> {0} - ERROR')
                    py_vals.append(0)
                py_dict = dict(zip(py_var, py_vals))
                df = pd.read_csv("data/case_study_results.csv")
                df = df[df.case_study == train['case_study']]
                df = df[df.scenario == train['scenario']]
                df = df[df.unit_process == unit]
                num = df[df.Variable == ex].value.max()
                #print(f'Excel --> {num}\n\n')
                excel_vals.append(num)
                excel_dict = dict(zip(py_var, excel_vals))
            big_dict[unit]['python'] = py_dict
            big_dict[unit]['excel'] = excel_dict
    else:
        pass 



    df = pd.DataFrame.from_dict({(i,j): big_dict[i][j] 
                               for i in big_dict.keys() 
                               for j in big_dict[i].keys()},
                           orient='index')
    
    return df

In [ ]:
get_results_table()

In [ ]:
train = case_study_trains.train 
source_water = case_study_trains.source_water 
source_or_use = None
# getting the list of consituents with removal factors that are bigger than 0
df1 = pd.read_csv("data/constituent_removal.csv")
df1 = df1[df1.reference == train["reference"]]
df1 = df1[df1.case_study == train["case_study"]]
df1 = df1[df1.scenario == train["scenario"]]

list1 = df1[df1.value >=0].constituent.unique()

import importfile

# grabs inlet water information
# df = importfile.feedwater(
#     input_file="data/case_study_water_sources.csv",
#     reference = source_water["reference"], 
#     water_type = source_water["water_type"], 
#     case_study = source_water["case_study"],
#     scenario = source_water["scenario"])

df2 = pd.read_csv('data/case_study_water_sources.csv', index_col='variable')
df2 = df2[df2.reference == source_water['reference']]
df2 = df2[df2.water_type == source_water['water_type']]
df2 = df2[df2.case_study == source_water['case_study']]
# df = df[df.source_or_use == source_water['source_or_use']]
df2 = df2[df2.scenario == source_water['scenario']]
df2 = df2.set_index(df2.index)
df2['feedwater'] = df2.value
df2['SourceNodeName'] = 'source_node'
# gets list of consituents in inlet water
list2 = df2.index

# # combines list
final_list = [x for x in list1 if x in list2]
print(f'list1 = {list1}')
print(f'list2 = {list2}')
print(f'final_list = {final_list}')



In [ ]:
df1

In [ ]:
df2

In [ ]:
import case_study_trains
unit_process = 'ro_deep'
train = case_study_trains.train 
source_water = case_study_trains.source_water 

df = pd.read_csv("data/constituent_removal.csv")
df.case_study = np.where(df.case_study == "Default", train["case_study"], df.case_study)
df = df[df.reference == train["reference"]]
df = df[df.case_study == train["case_study"]]
df = df[df.scenario == train["scenario"]]
df = df[df.unit_process == unit_process]

removal_dict = {}
for constituent in df.constituent.unique():
    removal_dict[constituent] = df[df.constituent == constituent].value.max()

In [ ]:
df

In [ ]:
removal_dict

In [ ]:
import generate_constituent_list
df = pd.read_csv("data/water_recovery.csv")
case_study_name = case_study_trains.train["case_study"]
unit_process_type = 'microfiltration'
unit_process_name = unit_process_type
if unit_process_type == "reverse_osmosis": unit_process_type = "ro_deep"

if case_study_name in df[df.unit_process == unit_process_type].case_study:
    if "calculated" not in df[((df.unit_process == unit_process_type) & (df.case_study == case_study_name))].recovery.max():
        flow_recovery_factor = float(df[((df.unit_process == unit_process_type) & (df.case_study == case_study_name))].recovery)
        print(f'A = {flow_recovery_factor}\n')
        getattr(m.fs, unit_process_name).water_recovery.fix(flow_recovery_factor)
else:
    if "calculated" not in df[((df.unit_process == unit_process_type) & (df.case_study == "Default"))].recovery.max():
        flow_recovery_factor = float(df[((df.unit_process == unit_process_type) & (df.case_study == "Default"))].recovery)
        print(f'B = {flow_recovery_factor}\n')
        getattr(m.fs, unit_process_name).water_recovery.fix(flow_recovery_factor)

# Get constituent list and removal rates for this unit process
train_constituent_removal_factors = generate_constituent_list.get_removal_factors(unit_process_type)

for constituent_name in getattr(m.fs, unit_process_name).config.property_package.component_list:
    print(constituent_name)

    if constituent_name in train_constituent_removal_factors.keys():
        getattr(m.fs, unit_process_name).removal_fraction[:, constituent_name].fix(train_constituent_removal_factors[constituent_name])
    else:
        getattr(m.fs, unit_process_name).removal_fraction[:, constituent_name].fix(0)

In [ ]:
value(m.fs.microfiltration.water_recovery[0])

In [ ]:
train_constituent_removal_factors

In [ ]:
m.fs.microfiltration.display()

In [ ]:
df[df.unit_process == unit_process_type]

In [ ]:
df

In [ ]:
cost_range_list = []; #results will be inputted in this array
#up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.

#for value_change in pct_to_target1: # cycles through each value from MC range
for value_change in [0.4, 0.8]: #, 0.9]:

    # create and build model
    m = wt.watertap_setup(dynamic = False)
    m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

    m.fs.tri_media_filtration.water_recovery.fix(value_change)

    # set variable to MC value
    wt.run_water_tap(m)
    results_table = get_results_table(m, unit_process_name)
    cost_range_list.append(results_table.total_up_cost.sum())


In [ ]:
cost_range_list

In [ ]:
#### DO NOT USE THE BELOW ####

In [ ]:
import time
from multiprocessing import Pool
import multiprocessing

mu = 0.6
sigma = .1
num_reps = 50

input_list = np.random.normal(mu,sigma, size = num_reps) #, sigma, num_reps).round(4)

count, bins, ignored = plt.hist(input_list, 25, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
          linewidth=2, color='r')
plt.show()

### INPUT TO MODEL LIST: ### CAN BE AUTOMATED FOR USER TO LABEL THE VARIABLE. TOOD ###
no_of_proc = 4
list_final = []
for i in range(no_of_proc):
    part2 = len(input_list) / no_of_proc
    i2 = ((i+1)*part2)
    list1 = input_list[int(i*part2):int(i2)]
    list_final.append(list1)
    
    
def monte_run(list_final):
    print('goes in')

    up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.
    cost_range_list = []; #results will be inputted in this array

    #for value_change in pct_to_target1: # cycles through each value from MC range
    for value_change in list_final:

        # create and build model
        m = wt.watertap_setup(dynamic = False)
        m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

        getattr(m.fs, up_name).water_recovery.fix(value_change)

        # set variable to MC value
        result = wt.run_water_tap(m)
        results_table = get_results_table(m, unit_process_names)
        cost_range_list.append(results_table.total_up_cost.sum())


    return cost_range_list

startTime = time.time()

pool=Pool()
dfs = pool.map(monte_run, list_final) #SomeClass().preprocess_data()

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
####TO DO LOAD AND SAVE!!

In [ ]:
#### SAVE TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# wt.save_train(T, path)

In [ ]:
# #### LOAD TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# TT = wt.load_train(path)

In [ ]:
# wt.display.show_train(TT)